In [101]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Ridge, SGDRegressor, ElasticNetCV
from scipy.stats import boxcox
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import (StandardScaler, MinMaxScaler,
                                   PolynomialFeatures)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_predict, train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [102]:
df = pd.read_csv('train.csv', sep =';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,white
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,red
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,red
3,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,white
4,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,white


In [103]:
df1 = df.drop_duplicates()
df1

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,white
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,red
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,red
3,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,white
4,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,7.1,0.5,0.03,2.6,0.076,21,92,0.9975,3.50,0.60,9.8,5,red
6709,7.2,0.2,0.19,7.7,0.045,53,176,0.9958,3.17,0.38,9.5,5,white
6710,6.7,0.3,0.34,7.5,0.036,39,124,0.9912,2.99,0.32,12.4,8,white
6712,8.0,0.2,0.31,5.6,0.049,24,97,0.9930,3.10,0.42,10.9,5,white


In [104]:

one_hot_encoded_data = pd.get_dummies(df1, columns = ['type'])
one_hot_encoded_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type_red,type_white
0,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,0,1
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,1,0
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,1,0
3,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,0,1
4,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,0,1


In [122]:
y = one_hot_encoded_data['quality']
X = one_hot_encoded_data.drop(columns = ['quality'])

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

In [124]:
mm = MinMaxScaler()
poly = PolynomialFeatures(degree = 2, include_bias = False)
X_train_s = mm.fit_transform(X_train)
X_test_s = mm.transform(X_test)
X_train_poly = poly.fit_transform(X_train_s)
X_test_poly = poly.transform(X_test_s)

In [125]:
def rmse(ytrue, ypredicted):
    return np.sqrt(mean_squared_error(ytrue, ypredicted))

In [126]:
linearRegression = LinearRegression().fit(X_train_poly, y_train)
alphas = [0.005, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 80]

ridgeCV = RidgeCV(alphas=alphas, 
                  cv=4).fit(X_train_poly, y_train)
alphas2 = np.array([1e-5, 5e-5, 0.0001, 0.0005])
alphas_int = alphas2 * 100000
lassoCV = LassoCV(alphas=alphas2,
                  max_iter=10000,
                  cv=3).fit(X_train_poly, y_train)
l1_ratios = np.linspace(0.1, 0.9, 9)

elasticNetCV = ElasticNetCV(alphas=alphas2, 
                            l1_ratio=l1_ratios,
                            max_iter=10000).fit(X_train_poly, y_train)
linearRegression_rmse = rmse(y_test, linearRegression.predict(X_test_poly))
logisticRegression = LogisticRegression(random_state=42)
logisticRegression.fit(X_train_poly, y_train)

sgd_reg = SGDRegressor(eta0=0.01, max_iter=1000, tol=1e-3)
sgd_reg.fit(X_train_poly, y_train)

C:\Users\IT tools\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0158271131176662, tolerance: 0.25368699109414755
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\IT tools\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39.73014410879318, tolerance: 0.25368699109414755
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\IT tools\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.1492657939602395, tolerance: 0.25286459923664123
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\IT tools\anaconda3\lib\site-packages\sklearn\linear_model\_coordinat

SGDRegressor()

In [127]:
ridgeCV_rmse = rmse(y_test, ridgeCV.predict(X_test_poly))
lassoCV_rmse = rmse(y_test, lassoCV.predict(X_test_poly))
elasticNetCV_rmse = rmse(y_test, elasticNetCV.predict(X_test_poly))
logisticregression_rmse = rmse(y_test, logisticRegression.predict(X_test_poly))
sgd_rmse = rmse(y_test, sgd_reg.predict(X_test_poly))

In [128]:
rmse_vals = [linearRegression_rmse, ridgeCV_rmse, lassoCV_rmse, elasticNetCV_rmse, logisticregression_rmse, sgd_rmse]

labels = ['Linear', 'Ridge', 'Lasso', 'ElasticNet','logistic','sgd']

rmse_df = pd.Series(rmse_vals, index=labels).to_frame()
rmse_df.rename(columns={0: 'RMSE'}, inplace=1)
rmse_df

,RMSE
Linear,0.730662
Ridge,0.732722
Lasso,0.733974
ElasticNet,0.734493
logistic,0.797250
sgd,0.763352


In [100]:
model_parameters_dict = {
    'Linear': {'penalty': 'none'},
    'Lasso': {'penalty': 'l2',
           'alpha': lasso_cv.alpha_},
    'Ridge': {'penalty': 'l1',
           'alpha': 1.0 / ridgeCV_rmse},
    'ElasticNet': {'penalty': 'elasticnet', 
                   'alpha': elasticnet_cv.alpha_,
                   'l1_ratio': elasticnet_cv.l1_ratio_}}

new_rmses = {}
for modellabel, parameters in model_parameters_dict.items():
    # following notation passes the dict items as arguments
    if modellabel == 'Linear':
        model = LinearRegression()
    else:
        # Use double-asterisk (**) to unpack the dictionary
        model = SGDRegressor(**parameters)
    
    model.fit(X_train_poly, y_train)
    new_rmses[modellabel] = rmse(y_test, model.predict(X_test_poly))

# Assuming rmse_df is already defined
rmse_df['RMSE-SGD-learningrate'] = pd.Series(new_rmses)
rmse_df['RMSE-SGD-learningrate'] = pd.Series(new_rmses)
rmse_df

NameError: name 'lasso_cv' is not defined